In [4]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.webservice import AciWebservice 
from azureml.core.model import Model
from azureml.core import Workspace, Experiment


# ws = Workspace.get(name="capstone")
ws = Workspace.from_config()
# exp = Experiment(workspace=ws, name="udacity-project-hyperparam-bayes")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: capstone
Azure region: westeurope
Subscription id: 72f46e0e-1451-4b79-92cd-fc8f7797bda7
Resource group: test-Jesse


In [5]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

env = Environment.from_conda_specification(name = 'project1', file_path = '.azureml/project1.yml')
# env = Environment.get(workspace, "AzureML-Minimal").clone(env_name)

for pip_package in ["scikit-learn"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='score.py',
                                    environment=env)

from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

In [6]:
model = Model(ws, name='automl')
service = Model.deploy(workspace=ws,
                        name = 'automl',
                        models= [model],
                        inference_config=inference_config,
                        deployment_config=deployment_config
)

service.wait_for_deployment(True)
print(service.state)
print("scoring URI: " + service.scoring_uri)

NameError: name 'aci_config' is not defined

## Get testset

In [ ]:
train_url = "https://raw.githubusercontent.com/jvanelteren/housing/master/datasets/housing_after_preprocessing.csv"
train_x = pd.read_csv(train_url)
train_x.rename(columns={'Unnamed: 0':'Column1'}, inplace=True)
train_x = train_x.drop(columns=['y'])

from sklearn.impute import SimpleImputer
class DFSimpleImputer(SimpleImputer):
    # just like SimpleImputer, but retuns a df
    # this approach creates problems with the add_indicator=True, since more columns are returned
    # so don't set add_indicator to True
    def transform(self, X,y=None):
        return pd.DataFrame(super().transform(X),columns=X.columns) 
    def __repr__(self):
        return f'SimpleImputer'
imp = DFSimpleImputer(strategy='most_frequent')
imp = imp.fit(train_x)

In [ ]:
test_url = "https://raw.githubusercontent.com/jvanelteren/housing/master/datasets/final_test.csv"
test_x = pd.read_csv(test_url)
test_x.rename(columns={'Unnamed: 0':'Column1'}, inplace=True)
test_x = imp.transform(test_x)
test_x = test_x.astype(train_x.dtypes.to_dict())

In [ ]:
def submit(model, filename='submission.csv'):
    pred = model.predict(test_x)
    out = pd.DataFrame()
    out['Id']= list(range(1461, 2920))
    out['SalePrice'] = np.exp(pred)
    out.to_csv('./submissions/'+ filename, index=False)


In [ ]:
import pandas as pd
import json
import requests 

headers = {'Content-Type':'application/json'}

data = json.dumps(test_x)
# data = json.loads(data)
response = requests.post(service.scoring_uri, data=data, headers=headers)
print(response.status_code)
print(response.elapsed)
print(response.json())
